Install all required librabries for LLM.

**pypdf** - for reading pdf documents.

**Python-dotenv** reads key-value pairs from a .env file

**transformers** - is a Python library that makes downloading and training state-of-the-art ML models easy.

**llama-index** -  to ease the integration of extensive external knowledge/documents index  with external large language models.

**sentence-transformers** - provides an easy method to compute dense vector representations for sentences, paragraphs, and images.

**langchain** - Integrates custom knowlege LLM API for integrating and generation of response.

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install -q llama-index
!pip -q install sentence-transformers
!pip install -q langchain

We will utilize https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF model for this demo. For cuBLAS setup required for GPU, pls refer to https://python.langchain.com/docs/integrations/llms/llamacpp

In [7]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

Setup loggers

In [9]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Import different required libraries and functions needed for project.
1. **SimpleDirectoryReader** is used to read pdf document from a given folder. **VectorStoreIndex** isused to store indexed embedded content and **ServiceContext** is container used for liniking custom indexes with the LLM API integration for query.
2. **LlamaCPP** is python biding for **LlamaCPP** https://python.langchain.com/docs/integrations/llms/llamacpp
3. **messages_to_prompt**, **completion_to_prompt** are prompts used by LlamaCPP while interacting with query
4. **HuggingFaceEmbeddings** used for embedding content using genberal purpose text embedding model https://huggingface.co/thenlper/gte-large
5. **ServiceContext** links embedding model with LLM
6. **LangchainEmbedding** is wrapper embedding utilitizing HuggingFaceEmbeddings.

In [10]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import torch
from llama_index.llms import LlamaCPP
from llama_index.llms.utils import messages_to_prompt, completion_to_prompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding


Loads the pdf files from the /content/Data folder  

In [46]:
documents = SimpleDirectoryReader("/content/Data").load_data()

Initialize LlamaCPP llm model using mistral-7b-instruct-v0.1.Q4_K_M.gguf url

In [19]:

llm = LlamaCPP(
    model_url = 'https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_path = None,
    temperature = 0.1,
    max_new_tokens = 256,
    context_window = 2900,
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 4368.44


4167it [00:22, 188.50it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Create embeding model with HuggingFaceEmbeddings

In [43]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Create servicecontext for linking llm model and embeding model

In [44]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Convert the pdf documents text into vector embedding and store in VectorStore

In [47]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

All setup done, now you are ready to query the LLM with your custom data from pdf

Create a query engine and ask questions, LLM with respond with answers from document. Validate if the spefic text is coming from custom pdf document you uploaded/used.

In [50]:
query_engine = index.as_query_engine()

while (True) :
  question = input()
  if question == 'exit':
    break
  response = query_engine.query(question)
  print(response)


what is tumor?


Llama.generate: prefix-match hit


 A tumor is a growth of abnormal cells that can occur in various parts of the body, including the brain. Tumors can be benign or malignant, with malignant tumors being cancerous and having the potential to spread within the body. The characteristics of a tumor include unregulated growth, abnormal cells that grow into or around parts of the body, and interference with normal functioning. In the case of brain tumors, they can be located in a critical part of the brain and cause life-threatening damage if malignant.
what is brain tumor?


Llama.generate: prefix-match hit


 A brain tumor is a growth or mass of abnormal cells in the brain or spinal cord. It can be benign (slow-growing and usually has distinct borders) or malignant (rapid-growing, invasive, and life-threatening). Malignant brain tumors are sometimes called brain cancer, but since primary brain tumors rarely spread outside the brain and spinal cord, they do not exactly fit the general definition of cancer. Cancer is defined by unregulated growth of abnormal cells that grow into or around parts of the body and interfere with their normal functioning and spread to distant organs in the body. Brain tumors can be called malignant if they have the characteristics of cancer cells, are located in a critical part of the brain, and are causing life-threatening damage. Malignant brain tumors that are cancerous can spread within the brain and spine, but rarely spread to other parts of the body.
Brain tumors can be called malignant if they?


Llama.generate: prefix-match hit


 Brain tumors can be called malignant if they have the characteristics of cancer cells, are located in a critical part of the brain, and are causing life-threatening damage.
How tumors are named?


Llama.generate: prefix-match hit


 Tumors are named based on a classification system. Most medical centers now use the World Health Organization (WHO) classification system for this purpose.
exit
